## Цей міні проект створений з ціллю продемонструвати стиль та навички написання коду.
Демонстрацію буду робити на стандартному датасеті "House Prices" з сайту kaggle. Задача якого написати програму, яка буде оцінювати вартість будинку враховуючи дані які в ньому є характеристики.

P.S. Проект не містить всіх знань якими я володію, а призначений щоб у вас була моживість ближче познайомитися зі мною і сформувати якусь думку про мій підхід до роботи.

In [247]:
# Імпортуємо бібліотеки для роботи з даними та їх візуалізацією
import pandas as pd
import numpy as np

In [248]:
# download datasets
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

### Перший етап підготовка даних

In [252]:
# Поглянемо які у нас надні а саме типи даних і наявність пропусків
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [253]:
# Видаляємо колонки з великою кількістю пропусків
# І одразу признаки запишемо в змінні "X_train" та "X_test"
X_test = data_test.drop(['Alley','PoolQC','Fence','MiscFeature','Id'], axis=1)
X_train = data_train.drop(['Alley','PoolQC','Fence','MiscFeature','Id','SalePrice'], axis=1)

In [254]:
# Цюльову змінну позначимо як "y_train"
y_train = data_train.SalePrice
y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [255]:
# Заповнимо пропуски які залишилися найчастішими даними в колонках
for col in X_train.columns:
    try:
        X_train[col].fillna(X_train[col].mode()[0], inplace= True)
        X_test[col].fillna(X_test[col].mode()[0], inplace= True)
    except:
        pass

In [256]:
# Перевіримо чи залишилися у нас пропуски
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [257]:
# Перетворимо категорійні дані на числові
type_columns = pd.DataFrame(X_train.dtypes).reset_index().rename(columns={'index':'columns',0:'type'})
categories_columns = list(type_columns[type_columns['type']=='object']['columns'])

categories_data = X_train[categories_columns]
numeric_data = X_train.drop(categories_columns,axis=1)
dummies_data = pd.get_dummies(categories_data)
X_train = pd.concat([numeric_data,dummies_data], axis=1)

categories_data = X_test[categories_columns]
numeric_data = X_test.drop(categories_columns,axis=1)
dummies_data = pd.get_dummies(categories_data)
X_test = pd.concat([numeric_data,dummies_data], axis=1)

In [258]:
# Подивимось що у нас вийшло
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [259]:
X_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [260]:
# Бачимо що у Х_test не вистачає колонок. Знайдемо яких саме і додамо їх
clear_columns = list(set(X_train.columns) - set(X_test.columns))
X_test[clear_columns] = 0
X_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Exterior2nd_Other,RoofMatl_Metal,RoofMatl_Roll,Utilities_NoSeWa,Condition2_RRAe,Electrical_Mix,Condition2_RRAn,Condition2_RRNn,HouseStyle_2.5Fin,RoofMatl_ClyTile
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,0,0,0,0,0,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
# Перетворимо датасети в масиви щоб нормувати дані
X_test_np = np.array(X_test)
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

In [263]:
# Нормуємо дані для того щоб у нас не було великих викидів і кожен признак мав однаковий вплив
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_np)
X_test_scaled = scaler.transform(X_test_np)

### Другий етап створення та навчання моделі

In [264]:
# Імпортуємо RandomForestRegressor для регресі та GridSearchCV для пошуку найкращих праметрів
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [265]:
# Створимо модель для навчання
forest_model = RandomForestRegressor()

param = {'n_estimators':range(10,100,10), 'max_depth':range(1,20,2)}

CV_model = GridSearchCV(estimator=RandomForestRegressor(), 
                            param_grid=param,
                            cv=5, 
                            scoring='neg_root_mean_squared_error',
                            n_jobs=-1, 
                            verbose=10)

In [268]:
# Навчимо модель
CV_model.fit(X_train_scaled,y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(1, 20, 2),
                         'n_estimators': range(10, 100, 10)},
             scoring='neg_root_mean_squared_error', verbose=10)

In [271]:
# Отримаємо результат оцінки будинків з тестових даних
y_pred = CV_model.predict(X_test_scaled)
y_pred

array([119637.36344538, 145691.31944444, 183581.57913643, ...,
       158932.67222222, 108528.59545455, 224852.00555556])

In [272]:
# Завантажимо реальні ціни з тестового датасету
from sklearn.metrics import mean_squared_error
y_true = pd.read_csv('sample_submission.csv')
y_true = np.array(y_true.SalePrice)

In [276]:
# Порахуємо середню квадратичну помилку для оцінки результату
mean_squared_error(y_true,y_pred, squared=False)

67612.46972394205

### Створимо та навчимо ще одну модель для порівння і розуміння успішності результату

In [278]:
# Імпортуємо KNeighborsRegressor для порівняння з попередньою моделлю
from sklearn.neighbors import KNeighborsRegressor

In [280]:
# Створимо нову можель
KNR_model = KNeighborsRegressor()

KNR_param = {'n_neighbors':range(1,100)}

KNR_CV_model = GridSearchCV(estimator=KNeighborsRegressor(), 
                            param_grid=KNR_param,
                            cv=5, 
                            scoring='neg_root_mean_squared_error',
                            n_jobs=-1, 
                            verbose=10)

In [281]:
# Навчимо нашу нову модель
KNR_CV_model.fit(X_train_scaled,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': range(1, 100)},
             scoring='neg_root_mean_squared_error', verbose=10)

In [283]:
# Отримаємо оцінку житла по новій моделі
KNR_y_pred = KNR_CV_model.predict(X_test_scaled)

In [284]:
# Порахуємо середню квадратичну помилку для оцінки результату
mean_squared_error(y_true,KNR_y_pred, squared=False)

27058.123484797936

[CV 2/5; 1/99] START n_neighbors=1..............................................
[CV 2/5; 1/99] END ..........n_neighbors=1;, score=-46918.227 total time=   0.1s
[CV 3/5; 2/99] START n_neighbors=2..............................................
[CV 3/5; 2/99] END ..........n_neighbors=2;, score=-46897.161 total time=   0.0s
[CV 2/5; 3/99] START n_neighbors=3..............................................
[CV 2/5; 3/99] END ..........n_neighbors=3;, score=-41273.792 total time=   0.0s
[CV 5/5; 3/99] START n_neighbors=3..............................................
[CV 5/5; 3/99] END ..........n_neighbors=3;, score=-50765.898 total time=   0.0s
[CV 3/5; 4/99] START n_neighbors=4..............................................
[CV 3/5; 4/99] END ..........n_neighbors=4;, score=-45765.398 total time=   0.0s
[CV 1/5; 5/99] START n_neighbors=5..............................................
[CV 1/5; 5/99] END ..........n_neighbors=5;, score=-37367.479 total time=   0.0s
[CV 5/5; 5/99] START n_neigh

[CV 1/5; 1/99] START n_neighbors=1..............................................
[CV 1/5; 1/99] END ..........n_neighbors=1;, score=-43287.590 total time=   0.1s
[CV 2/5; 2/99] START n_neighbors=2..............................................
[CV 2/5; 2/99] END ..........n_neighbors=2;, score=-40343.565 total time=   0.0s
[CV 5/5; 2/99] START n_neighbors=2..............................................
[CV 5/5; 2/99] END ..........n_neighbors=2;, score=-52646.548 total time=   0.0s
[CV 4/5; 3/99] START n_neighbors=3..............................................
[CV 4/5; 3/99] END ..........n_neighbors=3;, score=-36906.505 total time=   0.0s
[CV 2/5; 4/99] START n_neighbors=4..............................................
[CV 2/5; 4/99] END ..........n_neighbors=4;, score=-43379.776 total time=   0.0s
[CV 3/5; 5/99] START n_neighbors=5..............................................
[CV 3/5; 5/99] END ..........n_neighbors=5;, score=-44605.798 total time=   0.0s
[CV 1/5; 6/99] START n_neigh

[CV 4/5; 1/99] START n_neighbors=1..............................................
[CV 4/5; 1/99] END ..........n_neighbors=1;, score=-41485.054 total time=   0.1s
[CV 4/5; 4/99] START n_neighbors=4..............................................
[CV 4/5; 4/99] END ..........n_neighbors=4;, score=-34463.481 total time=   0.0s
[CV 2/5; 5/99] START n_neighbors=5..............................................
[CV 2/5; 5/99] END ..........n_neighbors=5;, score=-43826.109 total time=   0.0s
[CV 2/5; 6/99] START n_neighbors=6..............................................
[CV 2/5; 6/99] END ..........n_neighbors=6;, score=-42928.825 total time=   0.0s
[CV 4/5; 6/99] START n_neighbors=6..............................................
[CV 4/5; 6/99] END ..........n_neighbors=6;, score=-34144.277 total time=   0.0s
[CV 3/5; 7/99] START n_neighbors=7..............................................
[CV 3/5; 7/99] END ..........n_neighbors=7;, score=-44305.421 total time=   0.0s
[CV 4/5; 7/99] START n_neigh

### Бачимо що друга можель справилася краще, тож візьмемо оцінку яку вона дала

In [230]:
# Створимо датасет з Id тестових даних і оцінки відповідного житла
pred = pd.DataFrame({'Id':data_test['Id'],'SalePrice':KNR_y_pred}).set_index('Id')

In [231]:
# Запишемо дані в файл csv
pred.to_csv('Prediction of price')